In [1]:
import os, sys
sys.path.append('../')
from xd import *
from run_DMASS_Y3 import *
from utils import *
import esutil
import healpy as hp
from systematics import *
%matplotlib inline

%load_ext autoreload
%autoreload 2

In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
# Calling Gold mask and calculating map area
# calling map 

path = '/fs/scratch/PCON0003/warner785/bwarner/'
##GoldMask = callingEliGoldMask()
LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
ringhp = hp.nest2ring(4096, [LSSGoldmask['PIXEL']])
ind_good_ring = ringhp

print(LSSGoldmask.dtype)
##LSSMask = callingOldY1BAOLSSMask()

###LSSMask = LSSMask[LSSMask['DEC']>-3.0 ]
#GoldMask_st82 = Cuts.SpatialCuts(GoldMask, ra=320, ra2=360, dec=-2, dec2=2)
'''
GoldMask_st82 = LSSGoldmask[ LSSGoldmask['DEC'] > -3.0 ]

GoldMask_spt = LSSGoldMask[ LSSGoldMask['DEC'] < -3.0 ]
#GoldMask_spt = Cuts.SpatialCuts(GoldMask_spt, ra=0, ra2 = 100, dec=-52, dec2 = -48)

pixarea = hp.nside2pixarea( 4096, degrees = True)
sptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )
st82npix =  GoldMask_st82['PIXEL'].size # hp.get_map_size( GoldMask_st82 )
SPTMaparea = pixarea * sptnpix
ST82Maparea = pixarea * st82npix '''

[('PIXEL', '>i4'), ('SIGNAL', '>f4')]


"\nGoldMask_st82 = LSSGoldmask[ LSSGoldmask['DEC'] > -3.0 ]\n\nGoldMask_spt = LSSGoldMask[ LSSGoldMask['DEC'] < -3.0 ]\n#GoldMask_spt = Cuts.SpatialCuts(GoldMask_spt, ra=0, ra2 = 100, dec=-52, dec2 = -48)\n\npixarea = hp.nside2pixarea( 4096, degrees = True)\nsptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )\nst82npix =  GoldMask_st82['PIXEL'].size # hp.get_map_size( GoldMask_st82 )\nSPTMaparea = pixarea * sptnpix\nST82Maparea = pixarea * st82npix "

In [3]:
rootdir = '../output/sfd_train_photo_SamPle_nozband_ran10_3_full_tol1e5_validate/'

In [4]:
# calling BOSS cmass and applying dmass goodregion mask ----------------------------
#cmass = io.getSGCCMASSphotoObjcat()
train_sample = esutil.io.read('../output/sfd_train_photo_SamPle_nozband_ran10_3_full_tol1e5/train_sample_des.fits')
print('total num of train', train_sample.size)
print('\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------')
train_sample = Cuts.keepGoodRegion(train_sample)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('total num of train', train_sample.size)? (<ipython-input-4-6df596d910c9>, line 4)

In [ ]:
gold_st82 = io.SearchAndCallFits(path = '/n/des/lee.5922/data/gold_cat/', columns = None, keyword = 'Y1A1_GOLD_STRIPE82_v2')
gold_st82 = gold_st82[(gold_st82['MODEST_CLASS'] == 1)&(gold_st82['FLAGS_GOLD'] == 0 )]
gold_st82 = Cuts.keepGoodRegion(gold_st82)


gold_st82 = RemovingSLRReddening(gold_st82)
gold_st82 = AddingSFD98Reddening(gold_st82, kind='STRIPE82')

# flags and color cut
mask_all = priorCut_test(gold_st82)
gold_st82 = gold_st82[mask_all]

In [ ]:
healConfig={'out_nside':64, 'nest':True}

In [ ]:
train_sample = HealPixifyCatalogs(catalog=train_sample, healConfig=healConfig, ratag='RA', dectag = 'DEC')
gold_st82 = HealPixifyCatalogs(catalog=gold_st82, healConfig=healConfig, ratag='RA', dectag = 'DEC')

In [ ]:
evenmask_train = train_sample['HEALIndex']%2 == 0
evenmask = gold_st82['HEALIndex']%2 == 0

print 'odd', np.sum(evenmask_train), ' even', np.sum(~evenmask_train)

In [ ]:
GoldMask_st82_hpix64.size

In [ ]:
GoldMask_st82_hpix64 = HealPixifyCatalogs(catalog=GoldMask_st82, healConfig=healConfig, ratag='RA', dectag = 'DEC')
evenmask_map = GoldMask_st82_hpix64['HEALIndex']%2 == 0

#pixarea64 = hp.nside2pixarea( 64, degrees = True)

print 'odd', np.sum(evenmask_map), ' even', np.sum(~evenmask_map)
print 'area', 'odd', np.sum(evenmask_map)*pixarea, ' even', np.sum(~evenmask_map)*pixarea

In [ ]:
train_even = train_sample[evenmask_train]
train_odd = train_sample[~evenmask_train]
gold_st82_even = gold_st82[evenmask]
gold_st82_odd = gold_st82[~evenmask]

In [ ]:
rootdir2 = '../output/validate_NEST/'

In [ ]:
fitsio.write(rootdir2+'train_NEST_even.fits', train_even, clobber=True)
fitsio.write(rootdir2+'train_NEST_odd.fits', train_odd, clobber=True)

In [ ]:
fitsio.write(rootdir2+'gold_st82_NEST_even.fits', gold_st82_even, clobber=True)
fitsio.write(rootdir2+'gold_st82_NEST_odd.fits', gold_st82_odd, clobber=True)

In [ ]:
spatialcheck( [train_sample, train_even, train_odd], label = ['all', 'even NEST', 'odd NEST'] )

In [ ]:
mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82['RA'], \
                                     gold_st82['DEC'],2./3600, maxmatch=1)
cmass_mask = np.zeros(gold_st82.size, dtype=bool)
cmass_mask[mg2] = 1
clean_cmass_data_des, nocmass = gold_st82[cmass_mask], gold_st82[~cmass_mask]
print 'cmass_fraction even', clean_cmass_data_des.size *1./gold_st82.size
fraction_even = clean_cmass_data_des.size *1./gold_st82.size

In [ ]:
mg1, mg2, _ = esutil.htm.HTM(10).match(train_even['RA'], train_even['DEC'], gold_st82_even['RA'], \
                                     gold_st82_even['DEC'],2./3600, maxmatch=1)
cmass_mask_even = np.zeros(gold_st82_even.size, dtype=bool)
cmass_mask_even[mg2] = 1
clean_cmass_data_des_even, nocmass_even = gold_st82_even[cmass_mask_even], gold_st82_even[~cmass_mask_even]
print 'cmass_fraction even', clean_cmass_data_des_even.size *1./gold_st82_even.size
fraction_even = clean_cmass_data_des_even.size *1./gold_st82_even.size

In [ ]:
nocmass_even.size, clean_cmass_data_des_even.size

In [ ]:
mg1, mg2, _ = esutil.htm.HTM(10).match(train_odd['RA'], train_odd['DEC'], gold_st82_odd['RA'], \
                                     gold_st82_odd['DEC'],2./3600, maxmatch=1)
cmass_mask_odd = np.zeros(gold_st82_odd.size, dtype=bool)
cmass_mask_odd[mg2] = 1
clean_cmass_data_des_odd, nocmass_odd = gold_st82_odd[cmass_mask_odd], gold_st82_odd[~cmass_mask_odd]
print 'cmass_fraction odd', clean_cmass_data_des_odd.size *1./gold_st82_odd.size
fraction_odd = clean_cmass_data_des_odd.size *1./gold_st82_odd.size